# Grid Data Analysis - Batches 

## Imports and Folder Setup

In [1]:
import os
import numpy as np

In [ ]:
# folder_path = '/content/drive/MyDrive/Grid_data_14/data'
folder_path = 'E:/L2RPN/Dreamer_V3_Implimentation/data-20241123T131314Z-001/data'

In [10]:
sample_data_path = 'E:/L2RPN/Dreamer_V3_Implimentation/data-20241123T131314Z-001/data/episode_800_data.npz'

## Initial File Structure Analysis

In [13]:
# Load with allow_pickle=True
data = np.load(sample_data_path, allow_pickle=True)

print("Arrays in the file:", data.files)

# Let's examine each array carefully
for arr_name in data.files:
    arr = data[arr_name]
    print(f"\nArray name: {arr_name}")
    try:
        print(f"Shape: {arr.shape}")
        print("Type:", type(arr))
        #print("Sample content:")
        #if arr.ndim == 0:  # Handle 0-d arrays
        #    print(arr.item())
        #else:
        #    print(arr[0])  # Print first element
    except:
        print("Could not display shape/content (might be a special data type)")

Arrays in the file: ['obs', 'action', 'obs_next', 'reward', 'done', 'steps']

Array name: obs
Shape: (10379, 467)
Type: <class 'numpy.ndarray'>

Array name: action
Shape: (10379,)
Type: <class 'numpy.ndarray'>

Array name: obs_next
Shape: (10379, 467)
Type: <class 'numpy.ndarray'>

Array name: reward
Shape: (10379,)
Type: <class 'numpy.ndarray'>

Array name: done
Shape: (10379,)
Type: <class 'numpy.ndarray'>

Array name: steps
Shape: (10379,)
Type: <class 'numpy.ndarray'>


## Basic Action Space and State Space Range Analysis

In [14]:
import numpy as np
from pathlib import Path
import collections

def analyze_grid2op_data(data_dir, batch_size=1000):
    """Analyze Grid2Op data files in batches"""

    
    action_stats = collections.defaultdict(int)
    state_ranges = {
        'min': None,
        'max': None,
        'action_types_seen': set()
    }
    
    npz_files = list(Path(data_dir).glob('*.npz'))
    total_transitions = 0
    
    for file_path in npz_files:
        # Load each file one at a time
        with np.load(file_path, allow_pickle=True) as data:
            n_steps = len(data['obs'])
            
            # Process in batches to save memory
            for start_idx in range(0, n_steps, batch_size):
                end_idx = min(start_idx + batch_size, n_steps)
                
                # Analyze actions in this batch
                batch_actions = data['action'][start_idx:end_idx]
                for action in batch_actions:
                    # Count action types/patterns
                    action_key = str(action)  # Modify based on your action structure
                    action_stats[action_key] += 1
                
                # Track observation space coverage
                batch_obs = data['obs'][start_idx:end_idx]
                if state_ranges['min'] is None:
                    state_ranges['min'] = np.min(batch_obs, axis=0)
                    state_ranges['max'] = np.max(batch_obs, axis=0)
                else:
                    state_ranges['min'] = np.minimum(state_ranges['min'], np.min(batch_obs, axis=0))
                    state_ranges['max'] = np.maximum(state_ranges['max'], np.max(batch_obs, axis=0))
                
                total_transitions += end_idx - start_idx
                
                # Print progress
                print(f"Processed {total_transitions} transitions...")

    return action_stats, state_ranges

# Basic usage statistics
def print_dataset_stats(action_stats, state_ranges):
    print(f"\nTotal unique actions seen: {len(action_stats)}")
    print("\nMost common actions:")
    most_common = sorted(action_stats.items(), key=lambda x: x[1], reverse=True)[:10]
    for action, count in most_common:
        print(f"Action: {action}, Count: {count}")
    
    print("\nState space ranges:")
    print(f"Min values: {state_ranges['min'][:5]}...")  # First 5 features
    print(f"Max values: {state_ranges['max'][:5]}...")

## Topology Pattern Analysis 

In [15]:
def analyze_topology_coverage(data_dir, batch_size=1000):
    """Analyze topology action patterns specifically"""
    topology_patterns = collections.defaultdict(int)
    
    for file_path in Path(data_dir).glob('*.npz'):
        with np.load(file_path, allow_pickle=True) as data:
            actions = data['action']
            
            # Process in batches
            for i in range(0, len(actions), batch_size):
                batch = actions[i:i+batch_size]
                
                # Analyze topology changes
                for action in batch:
                    # Extract topology change pattern
                    # (Modify based on your action structure)
                    pattern = extract_topology_pattern(action)
                    topology_patterns[pattern] += 1
    
    return topology_patterns

def extract_topology_pattern(action):
    """
    Extract meaningful pattern from topology action
    Modify based on your action structure
    """
    # Example implementation
    if hasattr(action, 'set_bus'):
        return tuple(action.set_bus)
    return str(action)

In [18]:
# 2. Run the main analysis
print("Starting analysis...")
action_stats, state_ranges = analyze_grid2op_data(folder_path, batch_size=1000)

# 3. Print basic statistics
print("\nPrinting basic dataset statistics...")
print_dataset_stats(action_stats, state_ranges)

Starting analysis...
Processed 1000 transitions...
Processed 2000 transitions...
Processed 3000 transitions...
Processed 4000 transitions...
Processed 5000 transitions...
Processed 6000 transitions...
Processed 7000 transitions...
Processed 8000 transitions...
Processed 9000 transitions...
Processed 10000 transitions...
Processed 10321 transitions...
Processed 11321 transitions...
Processed 12321 transitions...
Processed 13321 transitions...
Processed 14321 transitions...
Processed 15321 transitions...
Processed 16321 transitions...
Processed 17321 transitions...
Processed 18321 transitions...
Processed 19321 transitions...
Processed 20321 transitions...
Processed 20627 transitions...
Processed 21627 transitions...
Processed 22627 transitions...
Processed 23627 transitions...
Processed 24627 transitions...
Processed 25627 transitions...
Processed 26627 transitions...
Processed 27627 transitions...
Processed 28627 transitions...
Processed 29627 transitions...
Processed 30627 transitions

In [19]:
# 3. Print basic statistics
print("\nPrinting basic dataset statistics...")
print_dataset_stats(action_stats, state_ranges)


Printing basic dataset statistics...

Total unique actions seen: 179

Most common actions:
Action: 17, Count: 39875
Action: 65, Count: 39826
Action: 18, Count: 39740
Action: 141, Count: 39713
Action: 66, Count: 39693
Action: 155, Count: 39689
Action: 146, Count: 39689
Action: 39, Count: 39684
Action: 153, Count: 39681
Action: 110, Count: 39680

State space ranges:
Min values: [2019.0 1.0 1.0 0.0 0.0]...
Max values: [2019.0 2.0 31.0 23.0 55.0]...


## Grid2OP Env Action Space Check

In [35]:
import grid2op
from grid2op.Action import PlayableAction
import numpy as np

# Create the environment
env_name = "l2rpn_case14_sandbox"  # or your specific environment
env = grid2op.make(env_name)

# Analyze action space
print("Analyzing Grid2Op Case 14 topology:")
print("==================================")

# Get basic environment information
print(f"Number of substations: {env.n_sub}")
print(f"Number of lines: {env.n_line}")
print(f"Number of generators: {env.n_gen}")
print(f"Number of loads: {env.n_load}")

# Get action space info
print(f"Action space type: {type(env.action_space)}")
print(f"Number of possible actions: {env.action_space.n}")

Analyzing Grid2Op Case 14 topology:
Number of substations: 14
Number of lines: 20
Number of generators: 6
Number of loads: 11
Action space type: <class 'grid2op.Space.GridObjects.ActionSpace_l2rpn_case14_sandbox'>
Number of possible actions: 166


In [36]:
# For topology actions specifically
print("\nTopology action space:")
print(f"Number of substations: {env.n_sub}")
for sub_id in range(env.n_sub):
    # Get number of elements in each substation
    n_elements = int(env.sub_info[sub_id])
    print(f"Substation {sub_id}: {n_elements} elements")

# You can also use action_space methods to check
print("\nAction space details:")
print(env.action_space.size())  # Get size of action space
print(env.action_space.n_sub) 


Topology action space:
Number of substations: 14
Substation 0: 3 elements
Substation 1: 6 elements
Substation 2: 4 elements
Substation 3: 6 elements
Substation 4: 5 elements
Substation 5: 7 elements
Substation 6: 3 elements
Substation 7: 2 elements
Substation 8: 5 elements
Substation 9: 3 elements
Substation 10: 3 elements
Substation 11: 3 elements
Substation 12: 4 elements
Substation 13: 3 elements

Action space details:
166
14


In [38]:
class ActionConverter:
    def __init__(self, env) -> None:
        self.action_space = env.action_space
        self.env = env
        self.sub_mask = []
        self.init_sub_topo()
        self.init_action_converter()
    def init_sub_topo(self):
        self.subs = np.flatnonzero(self.action_space.sub_info)
        self.sub_to_topo_begin, self.sub_to_topo_end = [], [] # These lists will eventually store the starting and ending indices, respectively, for each actionable substation's topology data within the environment's overall topology information.
        idx = 0 # This variable will be used to keep track of the current position within the overall topology data
        for num_topo in self.action_space.sub_info: # The code can efficiently extract the relevant portion of the overall topology data that specifically applies to the given substation
            self.sub_to_topo_begin.append(idx)
            idx += num_topo
            self.sub_to_topo_end.append(idx)
    def init_action_converter(self):
        self.actions = [self.env.action_space({})]
        self.n_sub_actions = np.zeros(len(self.action_space.sub_info), dtype=int)
        for i, sub in enumerate(self.subs):
            # Generating Topology Actions
            topo_actions = self.action_space.get_all_unitary_topologies_set(self.action_space, sub) # retrieves all possible topology actions for the current substation using the get_all_unitary_topologies_set method of the action_space object
            self.actions += topo_actions  # Appends the topology actions for the current substation to the actions list.
            self.n_sub_actions[i] = len(topo_actions) # Stores the number of topology actions for the current substation in the n_sub_actions array
            self.sub_mask.extend(range(self.sub_to_topo_begin[sub], self.sub_to_topo_end[sub])) # Extends the sub_mask list with indices corresponding to the topologies of the current substation.
        self.sub_pos = self.n_sub_actions.cumsum()
        self.n = sum(self.n_sub_actions)
    def act(self, action:int):
        return self.actions[action]
    def action_idx(self, action):
        return self.actions.index(action)
    
ac = ActionConverter(env)
ac.n

178

## Assessing coverage metrics

In [16]:
import numpy as np
from collections import defaultdict

def calculate_action_distribution_entropy(action_stats):
    """
    Calculate the entropy of the action distribution to measure action space coverage.
    
    Parameters:
    action_stats (defaultdict): Dictionary mapping actions to their counts
    
    Returns:
    float: Entropy value. Higher value means more uniform distribution.
    """
    # Get total number of actions
    total_actions = sum(action_stats.values())
    
    # Calculate probability of each action
    probabilities = [count/total_actions for count in action_stats.values()]
    
    # Calculate entropy: -Σ p(x) * log(p(x))
    entropy = -sum(p * np.log(p) for p in probabilities if p > 0)
    
    return entropy

In [26]:
def calculate_uniformity(action_stats):
    """
    Calculate how uniform the action distribution is (0 to 1 scale)
    
    Parameters:
    action_stats (defaultdict): Dictionary mapping actions to their counts
    
    Returns:
    float: Uniformity score between 0 and 1
        - 1.0 means perfectly uniform (all actions appear equally often)
        - 0 means very skewed (one action dominates)
    """
    if not action_stats:
        return 0.0
        
    # Get counts
    counts = np.array(list(action_stats.values()))
    total_actions = sum(counts)
    n_unique_actions = len(counts)
    
    # Calculate ideal uniform probability
    ideal_prob = 1.0 / n_unique_actions
    
    # Calculate actual probabilities
    actual_probs = counts / total_actions
    
    # Calculate deviation from uniform distribution
    deviation = np.abs(actual_probs - ideal_prob).mean()
    
    # Convert to uniformity score (0 to 1)
    uniformity = 1.0 - deviation
    
    return uniformity

In [30]:
def assess_coverage_metrics(action_stats, state_ranges):
    """Assess if dataset provides good coverage"""
    # Calculate action space coverage
    total_actions = sum(action_stats.values())
    unique_actions = len(action_stats)
    action_entropy = calculate_action_distribution_entropy(action_stats)
    
    # Calculate state space coverage
    state_range_span = state_ranges['max'] - state_ranges['min']
    
    print(f"Action space metrics:")
    print(f"- Unique actions: {unique_actions}")
    print(f"- Action entropy: {action_entropy:.2f}")
    print(f"- Action distribution uniformity: {calculate_uniformity(action_stats):.2f}")
    
    return {
        'action_coverage': action_entropy,
        'state_space_coverage': np.mean(state_range_span)
    }

In [29]:
# 5. Assess coverage metrics
print("\nAssessing coverage metrics...")
coverage_metrics = assess_coverage_metrics(action_stats, state_ranges)

# 6. Print final results
print("\nFinal Coverage Metrics:")
print(f"Action coverage entropy: {coverage_metrics['action_coverage']:.2f}")
print(f"State space coverage: {coverage_metrics['state_space_coverage']:.2f}")


Assessing coverage metrics...
Action space metrics:
- Unique actions: 179
- Action entropy: 5.19
- Action distribution uniformity: 1.00

Final Coverage Metrics:
Action coverage entropy: 5.19
State space coverage: 164.79


# Initial Approach of Analysis 



## (Not Memory oprimised) 


In [5]:
def one_hot_encode(actions, num_actions):
    """
    Convert an array of actions into one-hot encoding.
    Args:
    - actions (np.array): Array of integer actions.
    - num_actions (int): Total number of possible actions (size of action space).
    Returns:
    - np.array: One-hot encoded actions, shape (num_samples, num_actions).
    """
    actions = np.array(actions)
    one_hot_actions = np.zeros((len(actions), num_actions), dtype=np.float32)
    one_hot_actions[np.arange(len(actions)), actions] = 1
    return np.array(one_hot_actions)

In [6]:
import yaml
class Config:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            if isinstance(value, dict):
                # Recursively convert dictionaries to Config objects
                setattr(self, key, Config(value))
            else:
                setattr(self, key, value)
    @classmethod
    def from_yaml(cls, file_path):
        with open(file_path, 'r') as file:
            data = yaml.safe_load(file)
        return cls(data)

In [9]:
config = Config.from_yaml('eda-config.yml')

In [10]:
import os
import numpy as np

def load_npz_files_from_folder(folder_path):
    """
    Loads observations, rewards, actions, dones, and next observations from multiple .npz files in a folder.

    Args:
        folder_path (str): Path to the folder containing .npz files.

    Returns:
        Tuple of concatenated numpy arrays: (observations, rewards, one_hot_actions, dones, next_observations)
    """
    all_observations = []
    all_rewards = []
    all_actions = []
    all_dones = []
    all_next_observations = []

    # Iterate through all .npz files in the folder
    for filename in sorted(os.listdir(folder_path)):  # Sort files if order matters
        if filename.endswith(".npz"):
            file_path = os.path.join(folder_path, filename)
            try:
                npz_data = np.load(file_path, allow_pickle=True)
                # Ensure the expected keys exist
                if {'obs', 'reward', 'action', 'done', 'obs_next'}.issubset(npz_data.keys()):
                    all_observations.append(npz_data['obs'])
                    all_rewards.append(npz_data['reward'])
                    all_actions.append(npz_data['action'])
                    all_dones.append(npz_data['done'])
                    all_next_observations.append(npz_data['obs_next'])
                else:
                    print(f"Skipping {filename}: Missing required keys.")
            except Exception as e:
                print(f"Error loading {filename}: {e}")

    # Concatenate all arrays along the first axis
    observations = np.concatenate(all_observations, axis=0)
    rewards = np.concatenate(all_rewards, axis=0)
    actions = np.concatenate(all_actions, axis=0)
    dones = np.concatenate(all_dones, axis=0)
    next_observations = np.concatenate(all_next_observations, axis=0)

    # Convert actions to one-hot encoding
    one_hot_actions = one_hot_encode(actions, config.action_dim)

    return observations, rewards, one_hot_actions, dones, next_observations


In [ ]:
# observations, rewards, one_hot_actions, dones, next_observations = load_npz_files_from_folder(folder_path)

In [11]:
def load_npz_files_from_folder(folder_path, batch_size=10):
    """
    Loads observations, rewards, actions, dones, and next observations from multiple .npz files in batches.

    Args:
        folder_path (str): Path to the folder containing .npz files.
        batch_size (int): Number of files to process at once.

    Returns:
        Data as numpy arrays, split into observations, rewards, actions, dones, and next observations.
    """
    all_observations = []
    all_rewards = []
    all_actions = []
    all_dones = []
    all_next_observations = []

    # Iterate through all .npz files in the folder in batches
    for filename in sorted(os.listdir(folder_path)):  # Sort files if order matters
        if filename.endswith(".npz"):
            file_path = os.path.join(folder_path, filename)
            try:
                npz_data = np.load(file_path, allow_pickle=True)
                # Ensure the expected keys exist
                if {'obs', 'reward', 'action', 'done', 'obs_next'}.issubset(npz_data.keys()):
                    all_observations.append(npz_data['obs'])
                    all_rewards.append(npz_data['reward'])
                    all_actions.append(npz_data['action'])
                    all_dones.append(npz_data['done'])
                    all_next_observations.append(npz_data['obs_next'])
                else:
                    print(f"Skipping {filename}: Missing required keys.")
            except Exception as e:
                print(f"Error loading {filename}: {e}")

    # Concatenate all arrays along the first axis
    observations = np.concatenate(all_observations, axis=0)
    rewards = np.concatenate(all_rewards, axis=0)
    actions = np.concatenate(all_actions, axis=0)
    dones = np.concatenate(all_dones, axis=0)
    next_observations = np.concatenate(all_next_observations, axis=0)

    return observations, rewards, actions, dones, next_observations


In [ ]:
# Initialize containers to hold full data
full_observations = []
full_rewards = []
full_actions = []
full_dones = []
full_next_observations = []

# Process data in batches
for batch_observations, batch_rewards, batch_actions, batch_dones, batch_next_observations in load_npz_files_from_folder(folder_path, batch_size=10):
    full_observations.append(batch_observations)
    full_rewards.append(batch_rewards)
    full_actions.append(batch_actions)
    full_dones.append(batch_dones)
    full_next_observations.append(batch_next_observations)

Error loading episode_215_data.npz: 
Error loading episode_216_data.npz: 
Error loading episode_217_data.npz: 
Error loading episode_218_data.npz: 
Error loading episode_219_data.npz: 
Error loading episode_21_data.npz: 
Error loading episode_220_data.npz: 
Error loading episode_221_data.npz: 
Error loading episode_222_data.npz: 
Error loading episode_223_data.npz: 
Error loading episode_224_data.npz: 
Error loading episode_225_data.npz: 
Error loading episode_226_data.npz: 
Error loading episode_227_data.npz: 
Error loading episode_228_data.npz: 
Error loading episode_229_data.npz: 
Error loading episode_22_data.npz: 
Error loading episode_230_data.npz: 
Error loading episode_231_data.npz: 
Error loading episode_232_data.npz: 
Error loading episode_233_data.npz: 
Error loading episode_234_data.npz: 
Error loading episode_235_data.npz: 
Error loading episode_236_data.npz: 
Error loading episode_237_data.npz: 
Error loading episode_238_data.npz: 
Error loading episode_239_data.npz: 
Err

In [ ]:
# Concatenate the full data
observations = np.concatenate(full_observations, axis=0)
rewards = np.concatenate(full_rewards, axis=0)
actions = np.concatenate(full_actions, axis=0)
dones = np.concatenate(full_dones, axis=0)
next_observations = np.concatenate(full_next_observations, axis=0)

# At this point, you have the full dataset, and you can proceed with the in-depth analysis.
print(f"Observations Shape: {observations.shape}")
print(f"Rewards Shape: {rewards.shape}")
print(f"Actions Shape: {actions.shape}")
print(f"Dones Shape: {dones.shape}")
print(f"Next Observations Shape: {next_observations.shape}")

In [ ]:
# Get basic statistics
reward_mean = np.mean(rewards)
reward_std = np.std(rewards)
action_mean = np.mean(actions)
action_std = np.std(actions)

# Additional statistics per observation dimension
obs_mean = np.mean(observations, axis=0)
obs_std = np.std(observations, axis=0)

print(f"Reward Mean: {reward_mean}, Reward Std: {reward_std}")
print(f"Action Mean: {action_mean}, Action Std: {action_std}")
print(f"Observation Mean: {obs_mean}, Observation Std: {obs_std}")


In [ ]:
# Action distribution
action_unique, action_counts = np.unique(actions, return_counts=True)
action_distribution = dict(zip(action_unique, action_counts))

# Visualize action distribution
import matplotlib.pyplot as plt
plt.bar(action_distribution.keys(), action_distribution.values())
plt.title("Action Distribution")
plt.xlabel("Action")
plt.ylabel("Count")
plt.show()


In [ ]:
# Time-based reward trend
plt.plot(rewards)
plt.title("Reward Trend Over Time")
plt.xlabel("Time Step (Index)")
plt.ylabel("Reward")
plt.show()

# Time-based action trend (e.g., frequencies of actions over time)
plt.plot(action_counts)
plt.title("Action Count Trend Over Time")
plt.xlabel("Time Step (Index)")
plt.ylabel("Action Count")
plt.show()
